In [2]:
import numpy as np

a = np.load("target.npz")

In [10]:
type(a)

numpy.lib.npyio.NpzFile

In [11]:
a["trial_arm_0_initial_ee_pos"]

array([[0.8978357 , 0.21710517, 0.29041286]])

In [5]:
""" Hyperparameters for PR2 trajectory optimization experiment. """
from __future__ import division

from datetime import datetime
import os.path
import numpy as np

In [6]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.gca(projection = '3d')

EE_POINTS = np.array([[0.02, -0.025, 0.05], [0.02, -0.025, -0.05],
                      [0.02, 0.05, 0.0], [0,0,0]])

data_x = [EE_POINTS[i][0] for i in range(4) ] 
data_y = [EE_POINTS[i][1] for i in range(4) ] 
data_z = [EE_POINTS[i][2] for i in range(4) ] 

ax.scatter(data_x,data_y,data_z)

plt.show

ValueError: Unknown projection '3d'

<Figure size 432x288 with 0 Axes>

In [54]:
data_x = [EE_POINTS[i][0] for i in range(4) ] 
data_x

[0.02, 0.02, 0.02, 0.0]

In [7]:
from gps import __file__ as gps_filepath
from gps.agent.ros.agent_ros import AgentROS
from gps.algorithm.algorithm_traj_opt import AlgorithmTrajOpt
from gps.algorithm.cost.cost_fk import CostFK
from gps.algorithm.cost.cost_action import CostAction
from gps.algorithm.cost.cost_sum import CostSum
from gps.algorithm.cost.cost_utils import RAMP_LINEAR, RAMP_FINAL_ONLY
from gps.algorithm.dynamics.dynamics_lr_prior import DynamicsLRPrior
from gps.algorithm.dynamics.dynamics_prior_gmm import DynamicsPriorGMM
from gps.algorithm.traj_opt.traj_opt_lqr_python import TrajOptLQRPython
from gps.algorithm.policy.lin_gauss_init import init_lqr
from gps.gui.target_setup_gui import load_pose_from_npz
from gps.proto.gps_pb2 import JOINT_ANGLES, JOINT_VELOCITIES, \
        END_EFFECTOR_POINTS, END_EFFECTOR_POINT_VELOCITIES, ACTION, \
        TRIAL_ARM, AUXILIARY_ARM, JOINT_SPACE
from gps.utility.general_utils import get_ee_points
from gps.gui.config import generate_experiment_info

In [8]:
# ?
EE_POINTS = np.array([[0.02, -0.025, 0.05], [0.02, -0.025, -0.05],
                      [0.02, 0.05, 0.0]])

# sensor dimension 
SENSOR_DIMS = {
    JOINT_ANGLES: 7, 
    JOINT_VELOCITIES: 7,
    END_EFFECTOR_POINTS: 3 * EE_POINTS.shape[0],
    END_EFFECTOR_POINT_VELOCITIES: 3 * EE_POINTS.shape[0],
    ACTION: 7,
} 

# calibration
PR2_GAINS = np.array([3.09, 1.08, 0.393, 0.674, 0.111, 0.152, 0.098])

BASE_DIR = '/'.join(str.split(gps_filepath, '/')[:-2])
EXP_DIR = BASE_DIR + '/../experiments/pr2_example/'

x0s = []
ee_tgts = []
reset_conditions = []

common = {
    'experiment_name': 'my_experiment' + '_' + \
            datetime.strftime(datetime.now(), '%m-%d-%y_%H-%M'),
    'experiment_dir': EXP_DIR,
    'data_files_dir': EXP_DIR + 'data_files/',
    'target_filename': EXP_DIR + 'target.npz',
    'log_filename': EXP_DIR + 'log.txt',
    'conditions': 1,
}

# TODO(chelsea/zoe) : Move this code to a utility function
# Set up each condition.
for i in xrange(common['conditions']):

    ja_x0, ee_pos_x0, ee_rot_x0 = load_pose_from_npz(
        common['target_filename'], 'trial_arm', str(i), 'initial'
    )
    ja_aux, _, _ = load_pose_from_npz(
        common['target_filename'], 'auxiliary_arm', str(i), 'initial'
    )
    _, ee_pos_tgt, ee_rot_tgt = load_pose_from_npz(
        common['target_filename'], 'trial_arm', str(i), 'target'
    )

    x0 = np.zeros(32)
    x0[:7] = ja_x0
    x0[14:(14+3*EE_POINTS.shape[0])] = np.ndarray.flatten(
        get_ee_points(EE_POINTS, ee_pos_x0, ee_rot_x0).T
    )

    ee_tgt = np.ndarray.flatten(
        get_ee_points(EE_POINTS, ee_pos_tgt, ee_rot_tgt).T
    )

    aux_x0 = np.zeros(7)
    aux_x0[:] = ja_aux

    reset_condition = {
        TRIAL_ARM: {
            'mode': JOINT_SPACE,
            'data': x0[0:7],
        },
        AUXILIARY_ARM: {
            'mode': JOINT_SPACE,
            'data': aux_x0,
        },
    }

    x0s.append(x0)
    ee_tgts.append(ee_tgt)
    reset_conditions.append(reset_condition)


if not os.path.exists(common['data_files_dir']):
    os.makedirs(common['data_files_dir'])

agent = {
    'type': AgentROS,
    'dt': 0.05,
    'conditions': common['conditions'],
    'T': 100,
    'x0': x0s,
    'ee_points_tgt': ee_tgts,
    'reset_conditions': reset_conditions,
    'sensor_dims': SENSOR_DIMS,
    'state_include': [JOINT_ANGLES, JOINT_VELOCITIES, END_EFFECTOR_POINTS,
                      END_EFFECTOR_POINT_VELOCITIES],
    'end_effector_points': EE_POINTS,
    'obs_include': [],
}

algorithm = {
    'type': AlgorithmTrajOpt,
    'conditions': common['conditions'],
    'iterations': 10,
}

algorithm['init_traj_distr'] = {
    'type': init_lqr,
    'init_gains':  1.0 / PR2_GAINS,
    'init_acc': np.zeros(SENSOR_DIMS[ACTION]),
    'init_var': 1.0,
    'stiffness': 0.5,
    'stiffness_vel': 0.25,
    'final_weight': 50,
    'dt': agent['dt'],
    'T': agent['T'],
}

torque_cost = {
    'type': CostAction,
    'wu': 5e-3 / PR2_GAINS,
}

fk_cost1 = {
    'type': CostFK,
    # Target end effector is subtracted out of EE_POINTS in ROS so goal
    # is 0.
    'target_end_effector': np.zeros(3 * EE_POINTS.shape[0]),
    'wp': np.ones(SENSOR_DIMS[END_EFFECTOR_POINTS]),
    'l1': 0.1,
    'l2': 0.0001,
    'ramp_option': RAMP_LINEAR,
}

fk_cost2 = {
    'type': CostFK,
    'target_end_effector': np.zeros(3 * EE_POINTS.shape[0]),
    'wp': np.ones(SENSOR_DIMS[END_EFFECTOR_POINTS]),
    'l1': 1.0,
    'l2': 0.0,
    'wp_final_multiplier': 10.0,  # Weight multiplier on final timestep.
    'ramp_option': RAMP_FINAL_ONLY,
}

algorithm['cost'] = {
    'type': CostSum,
    'costs': [torque_cost, fk_cost1, fk_cost2],
    'weights': [1.0, 1.0, 1.0],
}

algorithm['dynamics'] = {
    'type': DynamicsLRPrior,
    'regularization': 1e-6,
    'prior': {
        'type': DynamicsPriorGMM,
        'max_clusters': 20,
        'min_samples_per_cluster': 40,
        'max_samples': 20,
    },
}

algorithm['traj_opt'] = {
    'type': TrajOptLQRPython,
}

algorithm['policy_opt'] = {}

config = {
    'iterations': algorithm['iterations'],
    'common': common,
    'verbose_trials': 0,
    'agent': agent,
    'gui_on': True,
    'algorithm': algorithm,
    'num_samples': 5,
}

common['info'] = generate_experiment_info(config)

In [38]:
 reset_condition = {
        TRIAL_ARM: {
            'mode': JOINT_SPACE,
            'data': x0[0:7],
        },
        AUXILIARY_ARM: {
            'mode': JOINT_SPACE,
            'data': aux_x0,
        },
    }

In [9]:
    ja_x0, ee_pos_x0, ee_rot_x0 = load_pose_from_npz(
        common['target_filename'], 'trial_arm', str(i), 'initial'
    )
    ja_aux, _, _ = load_pose_from_npz(
        common['target_filename'], 'auxiliary_arm', str(i), 'initial'
    )
    _, ee_pos_tgt, ee_rot_tgt = load_pose_from_npz(
        common['target_filename'], 'trial_arm', str(i), 'target'
    )


In [60]:
    ja_x0, ee_pos_x0, ee_rot_x0 = load_pose_from_npz(
        '/home/mmmil-hw/gps/python/../experiments/pr2_example/target.npz', 'trial_arm', str(1), 'initial'
    )

In [72]:
print ee_pos_x0.reshape
print ee_pos_x0

(1, 3)
[[0.8978357  0.21710517 0.29041286]]


In [83]:
ja_x0.shape

(1, 7)

In [7]:
3*EE_POINTS.shape[0]

9

In [32]:
x0[14:(14+3*EE_POINTS.shape[0])] = np.ndarray.flatten(
        get_ee_points(EE_POINTS, ee_pos_x0, ee_rot_x0).T
    )

In [34]:
ee_tgt

array([ 0.88628752, -0.04838788, -0.20061039,  0.81353489, -0.11597948,
       -0.21237605,  0.83137455, -0.05082013, -0.27205022])

In [33]:
x0

array([  0.40493573,  -0.26709164,   0.9860822 ,  -0.58687217,
       -30.29001274,  -0.59461712,  19.50285032,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.93170456,   0.22415112,
         0.24216087,   0.92470182,   0.2301047 ,   0.34173755,
         0.88712438,   0.16438409,   0.29281173,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])

In [10]:
ja_x0

array([[  0.40493573,  -0.26709164,   0.9860822 ,  -0.58687217,
        -30.29001274,  -0.59461712,  19.50285032]])

In [18]:
get_ee_points(EE_POINTS, ee_pos_x0, ee_rot_x0)

array([[[0.93170456, 0.92470182, 0.88712438],
        [0.22415112, 0.2301047 , 0.16438409],
        [0.24216087, 0.34173755, 0.29281173]]])

In [23]:
ee_pos_x0

array([[0.8978357 , 0.21710517, 0.29041286]])

In [29]:
ee_rot_x0.dot(EE_POINTS.T)

array([[[ 0.03386886,  0.02686612, -0.01071132],
        [ 0.00704595,  0.01299953, -0.05272109],
        [-0.048252  ,  0.05132469,  0.00239886]]])

In [28]:
ee_rot_x0.dot(EE_POINTS.T)+ee_pos_x0.T

array([[[0.93170456, 0.92470182, 0.88712438],
        [0.22415112, 0.2301047 , 0.16438409],
        [0.24216087, 0.34173755, 0.29281173]]])

In [1]:
    ja_x0, ee_pos_x0, ee_rot_x0 = load_pose_from_npz(
        common['target_filename'], 'trial_arm', str(i), 'initial'
    )
    ja_aux, _, _ = load_pose_from_npz(
        common['target_filename'], 'auxiliary_arm', str(i), 'initial'
    )
    _, ee_pos_tgt, ee_rot_tgt = load_pose_from_npz(
        common['target_filename'], 'trial_arm', str(i), 'target'
    )


NameError: name 'load_pose_from_npz' is not defined

In [35]:
reset_conditions

[{0: {'data': array([  0.40493573,  -0.26709164,   0.9860822 ,  -0.58687217,
          -30.29001274,  -0.59461712,  19.50285032]), 'mode': 1},
  1: {'data': array([0., 0., 0., 0., 0., 0., 0.]), 'mode': 1}}]

In [40]:
x0s

[array([  0.40493573,  -0.26709164,   0.9860822 ,  -0.58687217,
        -30.29001274,  -0.59461712,  19.50285032,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.93170456,   0.22415112,
          0.24216087,   0.92470182,   0.2301047 ,   0.34173755,
          0.88712438,   0.16438409,   0.29281173,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ])]

In [56]:
config

{'agent': {'T': 100,
  'conditions': 1,
  'dt': 0.05,
  'ee_points_tgt': [array([ 0.88628752, -0.04838788, -0.20061039,  0.81353489, -0.11597948,
          -0.21237605,  0.83137455, -0.05082013, -0.27205022])],
  'end_effector_points': array([[ 0.02 , -0.025,  0.05 ],
         [ 0.02 , -0.025, -0.05 ],
         [ 0.02 ,  0.05 ,  0.   ]]),
  'obs_include': [],
  'reset_conditions': [{0: {'data': array([  0.40493573,  -0.26709164,   0.9860822 ,  -0.58687217,
            -30.29001274,  -0.59461712,  19.50285032]), 'mode': 1},
    1: {'data': array([0., 0., 0., 0., 0., 0., 0.]), 'mode': 1}}],
  'sensor_dims': {0: 7, 1: 7, 2: 7, 3: 9, 4: 9},
  'state_include': [1, 2, 3, 4],
  'type': gps.agent.ros.agent_ros.AgentROS,
  'x0': [array([  0.40493573,  -0.26709164,   0.9860822 ,  -0.58687217,
          -30.29001274,  -0.59461712,  19.50285032,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.93170456,   0.22415112,
        

In [ ]:
{'agent':
 {'T': 100,
  'conditions': 1,
  'dt': 0.05,
  'ee_points_tgt': [array([ 0.88628752, -0.04838788, -0.20061039,  0.81353489, -0.11597948,
          -0.21237605,  0.83137455, -0.05082013, -0.27205022])],
  'end_effector_points': array([[ 0.02 , -0.025,  0.05 ],
         [ 0.02 , -0.025, -0.05 ],
         [ 0.02 ,  0.05 ,  0.   ]]),
  'obs_include': [],
  'reset_conditions': [{0: {'data': array([  0.40493573,  -0.26709164,   0.9860822 ,  -0.58687217,
            -30.29001274,  -0.59461712,  19.50285032]), 'mode': 1},   1: {'data': array([0., 0., 0., 0., 0., 0., 0.]), 'mode': 1}}],
  'sensor_dims': {0: 7, 1: 7, 2: 7, 3: 9, 4: 9},
  'state_include': [1, 2, 3, 4],
  'type': gps.agent.ros.agent_ros.AgentROS,
  'x0': [array([  0.40493573,  -0.26709164,   0.9860822 ,  -0.58687217,
          -30.29001274,  -0.59461712,  19.50285032,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.93170456,   0.22415112,
            0.24216087,   0.92470182,   0.2301047 ,   0.34173755,
            0.88712438,   0.16438409,   0.29281173,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   0.        ,   0.        ])]},
 'algorithm': {'conditions': 1, 'cost': {'costs': [{'type': gps.algorithm.cost.cost_action.CostAction,
     'wu': array([0.00161812, 0.00462963, 0.01272265, 0.0074184 , 0.04504505,
            0.03289474, 0.05102041])},
    {'l1': 0.1,
     'l2': 0.0001,
     'ramp_option': 2,
     'target_end_effector': array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
     'type': gps.algorithm.cost.cost_fk.CostFK,
     'wp': array([1., 1., 1., 1., 1., 1., 1., 1., 1.])},
    {'l1': 1.0,
     'l2': 0.0,
     'ramp_option': 4,
     'target_end_effector': array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
     'type': gps.algorithm.cost.cost_fk.CostFK,
     'wp': array([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
     'wp_final_multiplier': 10.0}],
   'type': gps.algorithm.cost.cost_sum.CostSum,
   'weights': [1.0, 1.0, 1.0]},
  'dynamics': {'prior': {'max_clusters': 20,
    'max_samples': 20,
    'min_samples_per_cluster': 40,
    'type': gps.algorithm.dynamics.dynamics_prior_gmm.DynamicsPriorGMM},
   'regularization': 1e-06,
   'type': gps.algorithm.dynamics.dynamics_lr_prior.DynamicsLRPrior},
  'init_traj_distr': {'T': 100,
   'dt': 0.05,
   'final_weight': 50,
   'init_acc': array([0., 0., 0., 0., 0., 0., 0.]),
   'init_gains': array([ 0.3236246 ,  0.92592593,  2.54452926,  1.48367953,  9.00900901,
           6.57894737, 10.20408163]),
   'init_var': 1.0,
   'stiffness': 0.5,
   'stiffness_vel': 0.25,
   'type': <function gps.algorithm.policy.lin_gauss_init.init_lqr>},
  'iterations': 10,
  'policy_opt': {},
  'traj_opt': {'type': gps.algorithm.traj_opt.traj_opt_lqr_python.TrajOptLQRPython},
  'type': gps.algorithm.algorithm_traj_opt.AlgorithmTrajOpt},
 'common': {'conditions': 1,
  'data_files_dir': '/home/mmmil-hw/gps/python/../experiments/pr2_example/data_files/',
  'experiment_dir': '/home/mmmil-hw/gps/python/../experiments/pr2_example/',
  'experiment_name': 'my_experiment_04-23-20_16-59',
  'info': 'exp_name:   my_experiment_04-23-20_16-59\nalg_type:   AlgorithmTrajOpt\nalg_dyn:    DynamicsLRPrior\nalg_cost:   CostSum(CostAction, CostFK, CostFK)\niterations: 10\nconditions: 1\nsamples:    5\n',
  'log_filename': '/home/mmmil-hw/gps/python/../experiments/pr2_example/log.txt',
  'target_filename': '/home/mmmil-hw/gps/python/../experiments/pr2_example/target.npz'},
 'gui_on': True,
 'iterations': 10,
 'num_samples': 5,
 'verbose_trials': 0}